In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [ ]:
# Definir las rutas de las carpetas de entrenamiento y validación
train_dir = '/content/drive/MyDrive/Dataset_entrenamiento'
val_dir = '/content/drive/MyDrive/Dataset_validacion'

# Preprocesamiento de datos con aumentación para el entrenamiento
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Cargar las imágenes desde las carpetas
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(240, 240), batch_size=16, class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(240, 240), batch_size=16, class_mode='binary')


In [ ]:
# Cargar el modelo EfficientNet-B1 preentrenado
base_model = tf.keras.applications.EfficientNetB1(weights='imagenet', include_top=False, input_shape=(240, 240, 3))

# Agregar capas personalizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='sigmoid')(x)  # Capa de salida para clasificación binaria

# Crear el modelo
model = Model(inputs=base_model.input, outputs=x)


In [ ]:
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Entrenar el modelo
history = model.fit(train_generator, epochs=10, validation_data=val_generator)


In [ ]:
# Evaluar el modelo con los datos de validación
val_generator.reset()
y_pred = model.predict(val_generator)
y_pred = np.where(y_pred > 0.5, 1, 0)  # Convertir las probabilidades a clases binarias
y_true = val_generator.classes

# Clasificación de reporte
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()

# Métricas: Precision, Recall, F1-score
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

# Curva ROC y AUC
fpr, tpr, thresholds = roc_curve(y_true, model.predict(val_generator).ravel())
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()
# Función para calcular el Dice Coefficient
def dice_coefficient(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (np.sum(y_true_f) + np.sum(y_pred_f))

# Calcular el Dice Coefficient
dice = dice_coefficient(y_true, y_pred)
print(f'Dice Coefficient: {dice}')
